In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder {folder_path} created successfully.")
    else:
        print(f"Folder {folder_path} already exists.")


create_folder_if_not_exists("plots")
create_folder_if_not_exists("errors")

pi = np.double(np.pi)
interval_start = -2 * pi
interval_end = pi
base_domain = np.arange(interval_start, interval_end, 0.00943)


def function_given(x):
    m = 2
    k = 2
    global pi
    return np.sin(m * x) * np.sin(k * (x * x) / pi)


def first_derivative(x):
    m = 2
    k = 2
    global pi
    return 2 * k * x * np.cos(k * x ** 2 / pi) * np.sin(m * x) / pi + m * np.sin(k * x ** 2 / pi) * np.cos(m * x)

**Równomierna dystrybucja**

In [ ]:
def even_domain(x_min, x_max, number_of_points):
    return np.linspace(x_min, x_max, number_of_points, dtype=np.double)

**Zera wielomianu Czebyszewa**

In [ ]:
def get_chebyshev_zeros(x_min=None, x_max=None, number_of_points=None):
    if x_min is None and x_max is not None or x_min is not None and x_max is None:
        raise ValueError("x_min and x_max must be both None or both not None")
    chebyshev_zeroes = np.array(0, dtype=np.double)
    for k in range(number_of_points):
        cur_zero = np.cos((2 * k + 1) * np.pi / (2 * number_of_points))
        if x_min is not None and x_max is not None:
            cur_zero = (x_max - x_min) / 2 * cur_zero + (x_max + x_min) / 2
        chebyshev_zeroes = np.append(chebyshev_zeroes, cur_zero)
    return chebyshev_zeroes[1:]

**Wizualizacja**

In [ ]:
def visualise(domain=None, nodes_x=None, nodes_y=None, f_y=None, Pn_x=None, title=None, x_size=8, y_size=6,
              plot_title=None,
              filename=None,
              show=False):
    '''
    :param domain: array of float64, of x coordinates of points to visualise
    :param nodes_x: array of x coordinates of nodes
    :param nodes_y: array of f(x) for each x in nodes_x
    :param f_y: array of f(x) for each x in domain
    :param Pn_x: array of Pn(x) for each x in domain
    :param title: a string
    :param x_size: size of a figure in x axis
    :param y_size: size of a figure in y axis
    :param plot_title: title of a plot
    :param filename: name of a file to save a figure
    :param show: if True, plt.show() will be called
    :return: None
    '''
    plt.figure(figsize=(x_size, y_size))
    if nodes_x is not None and nodes_y is not None:
        plt.scatter(nodes_x, nodes_y, label="Węzły", color="green")
    if f_y is not None:
        plt.plot(domain, f_y, label="Zadana funckcja", color="blue")

    if title is not None:
        plt.plot(domain, Pn_x, label=title, color="red")
    if plot_title is not None:
        plt.title(plot_title)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    if filename is not None:
        plt.savefig(f"plots/{filename}.png", dpi=300, format='png')
    if show:
        plt.show()

**Implementacja funkcji do laboratorium**

In [ ]:
function_given_values = []
for random_name in range(len(base_domain)):
    function_given_values.append(function_given(base_domain[random_name]))


def get_function_values(X):
    result = np.zeros(len(X), dtype=np.double)
    for i in range(len(X)):
        result[i] = function_given(X[i])
    return result


def get_abs_error(f1, f2):
    max_error = -1
    if len(f1) != len(f2):
        raise Exception("f1 and f2 must have the same length")
    for i in range(len(f1)):
        max_error = max(max_error, abs(f1[i] - f2[i]))
    return max_error


def get_square_mean_error(f1, f2):
    error_sum = 0
    if len(f1) != len(f2):
        raise Exception("f1 and f2 must have the same length")
    for i in range(len(f1)):
        error_sum += (f1[i] - f2[i]) ** 2
    return np.sqrt(error_sum) / len(f1)

**Aproksymacja średniokwadratowa wielomianami algebraicznymi**

In [ ]:
def trigonometric_approximation(function, domain, x_zeroes, poly_degree, input_weights=None):
    def scale_number(num, base_interval=(interval_start, interval_end), resultant_interval=(-pi, pi)):
        a_, b_ = base_interval
        c, d = resultant_interval
        scaled_num = ((num - a_) / (b_ - a_)) * (d - c) + c
        return scaled_num

    x_zeroes_scaled = [scale_number(x) for x in x_zeroes]
    a = []
    b = []
    n = len(x_zeroes_scaled)
    for i in range(n):
        a_i = 0
        b_i = 0
        for j in range(n):
            a_i += function(x_zeroes[j]) * np.cos(i * x_zeroes_scaled[j])
            b_i += function(x_zeroes[j]) * np.sin(i * x_zeroes_scaled[j])
        a.append(2 / n * a_i)
        b.append(2 / n * b_i)
    trigonometric_approximation_result = []
    print(a)
    print(b)
    for x in domain:
        x_scaled = scale_number(x)
        trigonometric_approximation_result.append(a[0] / 2)
        for j in range(1, poly_degree + 1):
            trigonometric_approximation_result[-1] += a[j] * np.cos(j * x_scaled) + b[j] * np.sin(j * x_scaled)
    return trigonometric_approximation_result


def visualise_approximation(number_of_nodes, domain, function, poly_degree, filename=None, show_plot=False):
    nodes_x = even_domain(x_min=interval_start, x_max=interval_end, number_of_points=number_of_nodes)
    nodes_y = []
    for i in range(len(nodes_x)):
        nodes_y.append(function(nodes_x[i]))
    approximation_result = trigonometric_approximation(function=function,
                                                       domain=domain,
                                                       x_zeroes=nodes_x,
                                                       poly_degree=poly_degree)
    visualise(domain=base_domain, nodes_x=nodes_x, nodes_y=nodes_y, f_y=function_given_values,
              Pn_x=approximation_result,
              title=f"Funkcja aproksymująca",
              plot_title=f"Aproksymacja średniokwadratowa wielomianami trygonometrycznymi dla {len(nodes_x)} węzłów i m = {poly_degree}",
              filename=filename,
              show=show_plot)

**Wykonanie ćwiczenia**

In [ ]:
visualise(base_domain, None, None, function_given_values, filename="funkcja")

In [ ]:
given_number_of_nodes = [5, 7, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]
given_poly_degree = [2, 3, 4, 5, 6, 8, 10, 12, 15, 20, 25, 30, 35]


def get_all_visualisations(number_cur, degree_cur):
    if degree > (number - 1) // 2:
        #print("Nie można wyznaczyć wielomianu o stopniu większym niż (n-1)/2")
        return

    print("Liczba węzłów: ", number_cur, " stopień wielomianu: ", degree_cur)
    visualise_approximation(number_of_nodes=number_cur,
                            domain=base_domain,
                            function=function_given,
                            poly_degree=degree_cur,
                            filename=f"n_{number_cur}_m_{degree_cur}",
                            show_plot=True)


for number in given_number_of_nodes:
    for degree in given_poly_degree:
        get_all_visualisations(number, degree)

In [ ]:
import csv


def save_to_csv(filename, data):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

In [ ]:
def approximation_max_results_errors():
    global given_number_of_nodes
    csv_header = ["n\m"]
    csv_file = [csv_header]
    for cur_degree in given_poly_degree:
        csv_header.append(cur_degree)
    for number_of_nodes in given_number_of_nodes:
        csv_record = None
        for cur_degree in given_poly_degree:
            if cur_degree > (number_of_nodes - 1) // 2:
                #print("Nie można wyznaczyć wielomianu o stopniu większym niż (n-1)/2")
                continue
            if cur_degree == given_poly_degree[0]:
                csv_record = [number_of_nodes]
            even_nodes = even_domain(x_min=interval_start, x_max=interval_end, number_of_points=number_of_nodes)
            exact_result = function_given_values

            poly_approximation_result = trigonometric_approximation(function=function_given,
                                                                    domain=base_domain,
                                                                    x_zeroes=even_nodes,
                                                                    poly_degree=cur_degree)
            poly_approximation_result_sqrt_error = get_abs_error(exact_result, poly_approximation_result)
            csv_record.append(poly_approximation_result_sqrt_error)
        csv_file.append(csv_record)
        save_to_csv("errors/trygo_approx_max_error.csv", csv_file)


approximation_max_results_errors()

In [ ]:
def approximation_sqrt_results_errors():
    global given_number_of_nodes
    csv_header = ["n\m"]
    csv_file = [csv_header]
    for cur_degree in given_poly_degree:
        csv_header.append(cur_degree)
    for number_of_nodes in given_number_of_nodes:
        csv_record = None
        for cur_degree in given_poly_degree:
            if cur_degree > (number_of_nodes - 1) // 2:
                #print("Nie można wyznaczyć wielomianu o stopniu większym niż (n-1)/2")
                continue
            if cur_degree == given_poly_degree[0]:
                csv_record = [number_of_nodes]
            even_nodes = even_domain(x_min=interval_start, x_max=interval_end, number_of_points=number_of_nodes)
            exact_result = function_given_values

            poly_approximation_result = trigonometric_approximation(function=function_given,
                                                                    domain=base_domain,
                                                                    x_zeroes=even_nodes,
                                                                    poly_degree=cur_degree)
            poly_approximation_result_sqrt_error = get_square_mean_error(exact_result, poly_approximation_result)
            csv_record.append(poly_approximation_result_sqrt_error)
        csv_file.append(csv_record)
        save_to_csv("errors/trygo_approx_sqrt_error.csv", csv_file)


approximation_sqrt_results_errors()